<a href="https://colab.research.google.com/github/ericttran3/coingecko-crypto-web-scraper/blob/master/colab/coingecko_web_scraper_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CoinGecko Web Scraper

This notebook is designed to teach you how to obtain data from top Cryptocurrency exchanges and API sources to build a trading bot for price arbitrage strategies.

- CoinGecko: https://github.com/man-c/pycoingecko



### Install Packages

In [ ]:
!pip install pycoingecko --quiet

### Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import os
import random
from random import randint
import time
from time import sleep

from datetime import datetime
from IPython.core.display import HTML, display

from pycoingecko import CoinGeckoAPI

from google.cloud import storage
from google.cloud import bigquery

### Configure Jupyter Notebook

In [ ]:
%matplotlib inline

pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_colwidth', 255)

display(HTML("<style>.container {width:85%} </style>"))

# Ingest Data

In [ ]:
# Instantiate API and store in alias
cg = CoinGeckoAPI()
cg.ping()

{'gecko_says': '(V3) To the Moon!'}

### Get Coins

In [ ]:
def get_coins(max_page):
    
    # Track how long python script takes to execute
    global df
    start = time.process_time()

    combined_list = []
    for page in range(1,max_page):
    
        try:
            coins_market = cg.get_coins_markets(vs_currency='usd', per_page=250, page=page, price_change_percentage='1h,24h,7d,14d,30d,200d,1y')
            combined_list += coins_market
            delay = randint(1, 2) 
            sleep(delay)
        except:
            pass
    
    # Create a dataframe from list of dictionaries
    df = pd.DataFrame(data=combined_list)
    df['created_date'] = datetime.today().strftime("%Y-%m-%d %H:%M:%S")
    print("Execution time: {} Seconds".format(round(time.process_time() - start)))
    
    return df

In [ ]:
# Maximum number of page as of 6/1/2021 is 31
get_coins(33)

Execution time: 1 Seconds


,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,price_change_percentage_14d_in_currency,price_change_percentage_1h_in_currency,price_change_percentage_1y_in_currency,price_change_percentage_200d_in_currency,price_change_percentage_24h_in_currency,price_change_percentage_30d_in_currency,price_change_percentage_7d_in_currency,created_date
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/large/bitcoin.png?1547033579,33113.00,6.187379e+11,1.0,6.937161e+11,4.878953e+10,35701.00,31109.000000,-2446.866501,-6.88103,-4.610657e+10,-6.93494,1.873028e+07,2.100000e+07,2.100000e+07,64805.00,-49.02496,2021-04-14T11:54:46.763Z,67.810000,48616.58672,2013-07-06T00:00:00.000Z,None,2021-06-08T19:44:04.454Z,-14.310002,1.906379,240.014958,85.820196,-6.881034,-43.658928,-11.322695,2021-06-08 19:46:23
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/large/ethereum.png?1595348880,2510.58,2.907343e+11,2.0,NaN,5.044825e+10,2731.14,2321.900000,-208.419269,-7.66530,-2.446353e+10,-7.76133,1.162108e+08,NaN,NaN,4356.99,-42.57545,2021-05-12T14:41:48.623Z,0.432979,577753.22700,2015-10-20T00:00:00.000Z,"{'times': 101.0027326057399, 'currency': 'btc', 'percentage': 10100.27326057399}",2021-06-08T19:44:47.199Z,-4.908085,1.718776,927.970281,432.671906,-7.665298,-35.835847,-7.305026,2021-06-08 19:46:23
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/large/Tether-logo.png?1598003707,1.01,6.319356e+10,3.0,NaN,1.082460e+11,1.01,0.977026,0.013408,1.34307,8.715291e+08,1.39843,6.246236e+10,6.246236e+10,NaN,1.32,-23.53489,2018-07-24T00:00:00.000Z,0.572521,76.71077,2015-03-02T00:00:00.000Z,None,2021-06-08T19:05:40.683Z,0.859893,0.580015,1.193494,1.130821,1.343072,1.287713,1.419064,2021-06-08 19:46:23
3,binancecoin,bnb,Binance Coin,https://assets.coingecko.com/coins/images/825/large/binance-coin-logo.png?1547034615,350.07,5.400958e+10,4.0,5.960159e+10,3.303656e+09,388.15,323.460000,-36.632888,-9.47311,-5.442949e+09,-9.15512,1.545337e+08,1.705337e+08,1.705337e+08,686.31,-49.07517,2021-05-10T07:24:17.097Z,0.039818,877651.58172,2017-10-19T00:00:00.000Z,None,2021-06-08T19:44:37.172Z,1.908423,1.514043,1913.570663,1148.183447,-9.473110,-45.730347,-0.852531,2021-06-08 19:46:23
4,cardano,ada,Cardano,https://assets.coingecko.com/coins/images/975/large/cardano.png?1547034860,1.56,4.979529e+10,5.0,6.987964e+10,5.485628e+09,1.68,1.410000,-0.114659,-6.86484,-3.629729e+09,-6.79406,3.206639e+10,4.500000e+10,4.500000e+10,2.45,-36.49994,2021-05-16T07:44:28.033Z,0.019253,7965.76280,2020-03-13T02:22:55.044Z,None,2021-06-08T19:45:05.046Z,0.805671,2.035239,1700.354286,1370.258488,-6.864838,-3.743735,-10.794939,2021-06-08 19:46:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7767,usd-open-dollar,usdo,USD Open Dollar,https://assets.coingecko.com/coins/images/14828/large/USDO-200px.png?1618562572,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000560e+09,NaN,NaN,0.00000,None,NaN,0.00000,None,None,2021-06-08T19:36:06.449Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-08 19:46:23
7768,complus-network,com,Complus Network,https://assets.coingecko.com/coins/images/14754/large/complus-logo-full.png?1618192114,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,None,NaN,0.00000,None,None,2021-05-26T18:29:15.161Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-08 19:46:23
7769,the-apis,api,The APIS,https://assets.coingecko.com/coins/images/14055/large/API.jpg?1614054286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,None,NaN,0.00000,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-08 19:46:23
7770,magic-e-stock,msb,Magic E-sto

In [ ]:
df.shape

(7772, 34)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7772 entries, 0 to 7771
Data columns (total 34 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   id                                        7772 non-null   object 
 1   symbol                                    7772 non-null   object 
 2   name                                      7772 non-null   object 
 3   image                                     7772 non-null   object 
 4   current_price                             7702 non-null   float64
 5   market_cap                                7704 non-null   float64
 6   market_cap_rank                           2829 non-null   float64
 7   fully_diluted_valuation                   943 non-null    float64
 8   total_volume                              7724 non-null   float64
 9   high_24h                                  6309 non-null   float64
 10  low_24h                             

In [ ]:
!pwd

/Users/etran/data-product/Projects/Web-Scrapers/coingecko-web-scraper/notebook


In [ ]:
df.to_csv('../data/coingecko-crypto-list_{}.csv'.format(datetime.today().strftime("%Y-%m-%d")), index=False)

### Get Coin Details

In [ ]:
columns = ['id', 'symbol', 'name', 'market_cap_rank']
coins_df = df[df['market_cap'] > 0][columns]
coins_df

,id,symbol,name,market_cap_rank
0,bitcoin,btc,Bitcoin,1.0
1,ethereum,eth,Ethereum,2.0
2,tether,usdt,Tether,3.0
3,binancecoin,bnb,Binance Coin,4.0
4,cardano,ada,Cardano,5.0
...,...,...,...,...
2824,argentum,arg,Argentum,2825.0
2825,smartcoin,smc,Smartcoin,2826.0
2826,peepcoin,pcn,Peepcoin,2827.0
2827,cstl,cstl,Castle,2828.0


In [ ]:
coins = list(coins_df['id'])
coins

['bitcoin',
 'ethereum',
 'tether',
 'binancecoin',
 'cardano',
 'dogecoin',
 'ripple',
 'usd-coin',
 'polkadot',
 'uniswap',
 'bitcoin-cash',
 'solana',
 'litecoin',
 'internet-computer',
 'chainlink',
 'binance-usd',
 'matic-network',
 'theta-token',
 'stellar',
 'vechain',
 'ethereum-classic',
 'wrapped-bitcoin',
 'filecoin',
 'tron',
 'eos',
 'dai',
 'monero',
 'aave',
 'okb',
 'shiba-inu',
 'compound-usd-coin',
 'kusama',
 'cdai',
 'neo',
 'compound-ether',
 'cosmos',
 'iota',
 'theta-fuel',
 'bitcoin-cash-sv',
 'pancakeswap-token',
 'celsius-degree-token',
 'tezos',
 'algorand',
 'maker',
 'crypto-com-chain',
 'ftx-token',
 'avalanche-2',
 'leo-token',
 'klay-token',
 'bittorrent-2',
 'huobi-token',
 'thorchain',
 'terra-luna',
 'amp-token',
 'safemoon',
 'terrausd',
 'hedera-hashgraph',
 'compound-governance-token',
 'sushi',
 'dash',
 'decred',
 'havven',
 'telcoin',
 'elrond-erd-2',
 'true-usd',
 'nem',
 'zcash',
 'yearn-finance',
 'waves',
 'holotoken',
 'zilliqa',
 'chiliz',

In [ ]:
len(coins)

2829

In [ ]:
# Track how long python script takes to execute
start = time.process_time()
timeout = time.time() + 3600 # 60 minutes

records = []

for coin in coins:
    coin_details = cg.get_coin_by_id(id=coin, vs_currency='usd')
    
    # Get id
    try:
        coin_id = coin_details['id']
    except:
        coin_id = ''
    
    # Get hashing algorith
    try:
        coin_algorithm = coin_details['hashing_algorithm']
    except:
        coin_algorithm = ''
        
    # Get coin categories
    try:
        coin_categories = coin_details['categories']
    except:
        coin_categories = ''
        
    # Get coin's country of origin
    try:
        coin_country_origin = coin_details['country_origin']
    except:
        coin_country_origin = ''
        
    # Get coin's gensis date
    try:
        coin_genesis_date = coin_details['genesis_date']
    except:
        coin_genesis_date = ''
        
    # Get coin's ICO start date
    try:
        coin_ico_start_date = coin_details['ico_data']['ico_start_date']
    except:
        coin_ico_start_date = ''
        
    # Get coin's ICO end date
    try:
        coin_ico_end_date = coin_details['ico_data']['ico_end_date']
    except:
        coin_ico_end_date = ''    
        
    # Get coin's short description
    try:
        coin_short_desc = coin_details['ico_data']['short_desc']
    except:
        coin_short_desc = ''
        
    # Get coins ico base sale amount
    try:
        coin_base_public_sale_amount = coin_details['ico_data']['base_public_sale_amount']
    except:
        coin_base_public_sale_amount = ''
        
    # Get coins ico quoted sale amount
    try:
        coin_quote_public_sale_amount = coin_details['ico_data']['quote_public_sale_amount']
    except:
        coin_quote_public_sale_amount = ''          
        
    # Get coin website
    try:
        coin_website = coin_details['links']['homepage']
    except:
        coin_website = ''
        
    # Get coin blockchain website
    try:
        blockchain_website = coin_details['links']['blockchain_site']
    except:
        blockchain_website = ''
        
    # Get coin forum
    try:
        coin_forum = coin_details['links']['official_forum_url']
    except:
        coin_forum = ''
        
    # Get Twitter handle
    try:
        twitter = coin_details['links']['twitter_screen_name']
    except:
        twitter = ''
        
    # Get Facebook handle
    try:
        facebook = coin_details['links']['facebook_username']
    except:
        facebook = ''
        
    # Get Reddit handle
    try:
        subreddit_url = coin_details['links']['subreddit_url']
    except:
        subreddit_url = ''
        
    # Social Media Community
    # Get twitter followers
    try:
        twitter_followers = coin_details['community_data']['twitter_followers']
    except:
        twitter_followers = ''
        
    # Get reddit average post 48 hours
    try:
        reddit_avg_post_48h = coin_details['community_data']['reddit_average_posts_48h']
    except:
        reddit_avg_post_48h = ''
        
    # Get reddit average comments 48 hours
    try:
        reddit_avg_comments_48h = coin_details['community_data']['reddit_average_posts_48h']
    except:        
        reddit_avg_comments_48h = ''
        
    # Get reddit subscribers
    try:
        reddit_subscribers = coin_details['community_data']['reddit_subscribers']
    except:
        reddit_subscribers = ''
        
    # Get reddit active accounts 48 hours
    try:
        reddit_active_accounts_48h = coin_details['community_data']['reddit_accounts_active_48h']
    except:
        reddit_active_accounts_48h = ''
    
    # Get Alexa rank
    try:
        alexa_rank = coin_details['public_interest_stats']['alexa_rank']
    except:
        alexa_rank = ''
        
    # Get Bing matches
    try:
        bing_matches = coin_details['public_interest_stats']['bing_matches']
    except:
        bing_matches = ''        
        
    # Get coin's sentiment data
    try:
        coin_sentiment = coin_details['sentiment_votes_up_percentage']
    except:
        coin_sentiment = ''
        
    # Get coingecko score
    try:
        coin_coingecko_score = coin_details['coingecko_score']
    except:
        coin_coingecko_score = ''
        
    # Get developer score
    try: 
        coin_developer_score = coin_details['developer_score']
    except:
        coin_developer_score = ''
        
    # Get community score
    try:
        coin_community_score = coin_details['community_score']
    except:
        coin_community_score = ''
        
    # Get liquidity score
    try:
        coin_liquidity_score = coin_details['liquidity_score']
    except:
        coin_liquidity_score = ''
        
    # Get public interest score
    try:
        coin_public_interest_score = coin_details['public_interest_score']
    except:
        coin_public_interest_score = ''
        
    
    # Developer Community
    # Get Github repos
    try:
        github_repos = coin_details['links']['repos_url']['github']
    except:
        github_repos = ''

    # Get github forks        
    try:
        github_forks = coin_details['developer_data']['forks']
    except:
        github_forks = ''
        
    # Get github stars
    try:
        github_stars = coin_details['developer_data']['stars']
    except:
        github_stars = ''
        
    # Get github subscribers
    try:
        github_subscribers = coin_details['developer_data']['subscribers']
    except:
        github_subscribers = ''
        
    # Get github issues
    try:
        github_issues = coin_details['developer_data']['total_issues']
    except:
        github_issues = ''
        
    # Get github closed issues
    try:
        github_closed_issues = coin_details['developer_data']['closed_issues']
    except:
        github_closed_issues = ''
        
    # Get github pulled requests merged
    try:
        github_pr_merged = coin_details['developer_data']['pull_requests_merged']
    except:
        github_pr_merged = ''
        
    # Get github pull requests contribuors
    try:
        github_pr_contributers = coin_details['developer_data']['pull_request_contributors']
    except:
        github_pr_contributers = ''
        
    # Get github developer additions
    try:
        github_additions_4w = coin_details['developer_data']['code_additions_deletions_4_weeks']['additions']
    except:
        github_additions_4w = ''
        
    # Get github developer deletions
    try:
        github_deletions_4w = coin_details['developer_data']['code_additions_deletions_4_weeks']['deletions']
    except:
        github_deletions_4w = ''
        
    # Get github commits in last 4 weeks
    try:
        github_commits_4w = coin_details['developer_data']['commit_count_4_weeks']
    except:
        github_commits_4w = ''
    
    record = (coin_id, coin_algorithm, coin_categories, coin_country_origin, 
              coin_genesis_date, coin_ico_start_date, coin_ico_end_date, coin_short_desc, coin_base_public_sale_amount, coin_quote_public_sale_amount, 
              coin_website, blockchain_website, coin_forum, twitter, facebook, twitter_followers, subreddit_url, reddit_avg_post_48h, reddit_avg_comments_48h, 
              reddit_subscribers, reddit_active_accounts_48h, alexa_rank, bing_matches, coin_sentiment, coin_coingecko_score, coin_developer_score,
              coin_community_score, coin_liquidity_score, coin_public_interest_score, github_repos, github_forks, github_stars, github_subscribers, 
              github_issues, github_closed_issues, github_pr_merged, github_pr_contributers, github_additions_4w, github_deletions_4w, github_commits_4w)

    records.append(record)
    delay = randint(1, 2) 
    sleep(delay) # Delay pagination by 1 to 2 seconds each time

print("Execution time: {} minutes".format(round(time.process_time() - start,2)))

Execution time: 34.89 minutes


In [ ]:
columns = ['coin_id', 'coin_algorithm', 'coin_categories', 'coin_country_origin', 'coin_genesis_date', 'coin_ico_start_date', 'coin_ico_end_date', 'coin_short_desc', 
           'coin_base_public_sale_amount', 'coin_quote_public_sale_amount', 'coin_website', 'blockchain_website', 'coin_forum', 'twitter', 'facebook', 'twitter_followers',
           'subreddit_url', 'reddit_avg_post_48h', 'reddit_avg_comments_48h', 'reddit_subscribers', 'reddit_active_accounts_48h', 'alexa_rank', 
           'coin_sentiment', 'coin_coingecko_score', 'coin_developer_score', 'coin_community_score', 'coin_liquidity_score', 'coin_public_interest_score', 'github_repos', 
           'github_forks', 'github_stars', 'github_subscribers', 'github_issues', 'github_closed_issues', 'github_pr_merged', 'github_pr_contributers', 
           'github_additions_4w', 'github_deletions_4w', 'github_commits_4w']
df_cd = pd.DataFrame(data=records, columns = columns)
df_cd

,coin_id,coin_algorithm,coin_categories,coin_country_origin,coin_genesis_date,coin_ico_start_date,coin_ico_end_date,coin_short_desc,coin_base_public_sale_amount,coin_quote_public_sale_amount,coin_website,blockchain_website,coin_forum,twitter,facebook,twitter_followers,subreddit_url,reddit_avg_post_48h,reddit_avg_comments_48h,reddit_subscribers,reddit_active_accounts_48h,alexa_rank,bing_matches,coin_sentiment,coin_coingecko_score,coin_developer_score,coin_community_score,coin_liquidity_score,coin_public_interest_score,github_repos,github_forks,github_stars,github_subscribers,github_issues,github_closed_issues,github_pr_merged,github_pr_contributers,github_additions_4w,github_deletions_4w,github_commits_4w
0,bitcoin,SHA-256,[Cryptocurrency],,2009-01-03,,,,,,"[http://www.bitcoin.org, , ]","[https://blockchair.com/bitcoin/, https://btc.com/, https://btc.tokenview.com/, , ]","[https://bitcointalk.org/, , ]",bitcoin,bitcoins,2682026,https://www.reddit.com/r/Bitcoin/,6.583,6.583,3054626,9452,9440.0,None,52.41,81.317,98.883,75.318,100.038,0.381,"[https://github.com/bitcoin/bitcoin, https://github.com/bitcoin/bips]",28996,54527,3816,6274,5692,8678,729,5220.0,-5205.0,381
1,ethereum,Ethash,[Smart Contract Platform],,2015-07-30,2014-07-20T00:00:00.000Z,2014-09-01T00:00:00.000Z,A decentralized platform for applications,1,0.00074794,"[https://www.ethereum.org/, , ]","[https://etherscan.io/, https://ethplorer.io/, https://blockchair.com/ethereum, https://eth.tokenview.com/, https://hecoinfo.com/token/0x64ff637fb478863b7468bc97d30a5bf3a428a1fd]","[https://forum.ethereum.org/, , ]",ethereum,ethereumproject,1346648,https://www.reddit.com/r/ethereum,6.727,6.727,999142,2659,8793.0,None,63.75,78.268,97.236,65.118,100.782,0.473,"[https://github.com/ethereum/go-ethereum, https://github.com/ethereum/py-evm, https://github.com/ethereum/aleth, https://github.com/ethereum/web3.py, https://github.com/ethereum/solidity, https://github.com/ethereum/sharding, https://github.com/ethere...",11276,30935,2108,5618,5423,3766,531,9935.0,-4824.0,58
2,tether,None,"[USD Stablecoin, Stablecoins]",,None,,,,,,"[https://tether.to/, , ]","[https://blockchair.com/bitcoin/omni/property/31, https://www.omniexplorer.info/asset/31, https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7, https://ethplorer.io/address/0xdac17f958d2ee523a2206206994597c13d831ec7, https://tronscan....","[, , ]",Tether_to,tether.to,122013,None,0.000,0.000,0,0,74251.0,None,68.78,42.054,0.000,10.771,107.145,0.066,[],0,0,0,0,0,0,0,NaN,NaN,0
3,binancecoin,None,"[Centralized Exchange Token (CEX), Binance Smart Chain Ecosystem, Exchange-based Tokens]",,2017-07-08,,,,,,"[https://www.binance.com/, , ]","[https://binance.mintscan.io/, https://explorer.binance.org/, https://bscscan.com, https://etherscan.io/token/0xB8c77482e45F1F44dE1745F52C74426C631bDD52, https://ethplorer.io/address/0xB8c77482e45F1F44dE1745F52C74426C631bDD52]","[, , ]",binance,binanceexchange,4185392,https://www.reddit.com/r/binance,9.364,9.364,480096,3305,888.0,None,65.06,68.888,73.255,64.872,86.742,14.394,"[https://github.com/binance-exchange/binance-official-api-docs, https://github.com/binance-exchange/node-binance-api, https://github.com/binance-exchange/php-binance-api]",2020,3405,498,122,122,55,26,0.0,0.0,0
4,cardano,None,[Smart Contract Platform],,None,,,,,,"[https://www.cardano.org/en/home/, , ]","[https://cardanoexplorer.com/, https://cardanoscan.io/, https://blockchair.com/cardano, https://adaex.org/, https://adastat.net/]","[, , ]",CardanoStiftung,,498080,https://www.reddit.com/r/cardano,6.083,6.083,509602,2220,42419.0,None,71.36,65.752,70.463,59.135,84.797,0.302,"[https://github.com/input-output-hk/cardano-sl, https://github.com/input-output-hk/js-cardano-wasm, https://github.com/input-output-hk/plutus, https://github.com/input-output-hk/rust-cardano, https://github.com/input-output-hk/cardano-chain]",608,3620,445,850,665,1706,80,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [ ]:
!pwd

/Users/etran/data-product/Projects/Web-Scrapers/coingecko-web-scraper/notebook


In [ ]:
# Save to local
df_cd.to_csv('../data/coingecko-crypto-coin-details_{}.csv'.format(datetime.today().strftime("%Y-%m-%d")), index=False)

### Combine Crypto Coins and Coin Details DataFrames

In [ ]:
df

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,price_change_percentage_14d_in_currency,price_change_percentage_1h_in_currency,price_change_percentage_1y_in_currency,price_change_percentage_200d_in_currency,price_change_percentage_24h_in_currency,price_change_percentage_30d_in_currency,price_change_percentage_7d_in_currency,created_date
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/large/bitcoin.png?1547033579,33113.00,6.187379e+11,1.0,6.937161e+11,4.878953e+10,35701.00,31109.000000,-2446.866501,-6.88103,-4.610657e+10,-6.93494,1.873028e+07,2.100000e+07,2.100000e+07,64805.00,-49.02496,2021-04-14T11:54:46.763Z,67.810000,48616.58672,2013-07-06T00:00:00.000Z,None,2021-06-08T19:44:04.454Z,-14.310002,1.906379,240.014958,85.820196,-6.881034,-43.658928,-11.322695,2021-06-08 19:46:23
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/large/ethereum.png?1595348880,2510.58,2.907343e+11,2.0,NaN,5.044825e+10,2731.14,2321.900000,-208.419269,-7.66530,-2.446353e+10,-7.76133,1.162108e+08,NaN,NaN,4356.99,-42.57545,2021-05-12T14:41:48.623Z,0.432979,577753.22700,2015-10-20T00:00:00.000Z,"{'times': 101.0027326057399, 'currency': 'btc', 'percentage': 10100.27326057399}",2021-06-08T19:44:47.199Z,-4.908085,1.718776,927.970281,432.671906,-7.665298,-35.835847,-7.305026,2021-06-08 19:46:23
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/large/Tether-logo.png?1598003707,1.01,6.319356e+10,3.0,NaN,1.082460e+11,1.01,0.977026,0.013408,1.34307,8.715291e+08,1.39843,6.246236e+10,6.246236e+10,NaN,1.32,-23.53489,2018-07-24T00:00:00.000Z,0.572521,76.71077,2015-03-02T00:00:00.000Z,None,2021-06-08T19:05:40.683Z,0.859893,0.580015,1.193494,1.130821,1.343072,1.287713,1.419064,2021-06-08 19:46:23
3,binancecoin,bnb,Binance Coin,https://assets.coingecko.com/coins/images/825/large/binance-coin-logo.png?1547034615,350.07,5.400958e+10,4.0,5.960159e+10,3.303656e+09,388.15,323.460000,-36.632888,-9.47311,-5.442949e+09,-9.15512,1.545337e+08,1.705337e+08,1.705337e+08,686.31,-49.07517,2021-05-10T07:24:17.097Z,0.039818,877651.58172,2017-10-19T00:00:00.000Z,None,2021-06-08T19:44:37.172Z,1.908423,1.514043,1913.570663,1148.183447,-9.473110,-45.730347,-0.852531,2021-06-08 19:46:23
4,cardano,ada,Cardano,https://assets.coingecko.com/coins/images/975/large/cardano.png?1547034860,1.56,4.979529e+10,5.0,6.987964e+10,5.485628e+09,1.68,1.410000,-0.114659,-6.86484,-3.629729e+09,-6.79406,3.206639e+10,4.500000e+10,4.500000e+10,2.45,-36.49994,2021-05-16T07:44:28.033Z,0.019253,7965.76280,2020-03-13T02:22:55.044Z,None,2021-06-08T19:45:05.046Z,0.805671,2.035239,1700.354286,1370.258488,-6.864838,-3.743735,-10.794939,2021-06-08 19:46:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7767,usd-open-dollar,usdo,USD Open Dollar,https://assets.coingecko.com/coins/images/14828/large/USDO-200px.png?1618562572,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000560e+09,NaN,NaN,0.00000,None,NaN,0.00000,None,None,2021-06-08T19:36:06.449Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-08 19:46:23
7768,complus-network,com,Complus Network,https://assets.coingecko.com/coins/images/14754/large/complus-logo-full.png?1618192114,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,None,NaN,0.00000,None,None,2021-05-26T18:29:15.161Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-08 19:46:23
7769,the-apis,api,The APIS,https://assets.coingecko.com/coins/images/14055/large/API.jpg?1614054286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,None,NaN,0.00000,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-08 19:46:23
7770,magic-e-stock,msb,Magic E-sto

In [ ]:
df_cd

,coin_id,coin_algorithm,coin_categories,coin_country_origin,coin_genesis_date,coin_ico_start_date,coin_ico_end_date,coin_short_desc,coin_base_public_sale_amount,coin_quote_public_sale_amount,coin_website,blockchain_website,coin_forum,twitter,facebook,twitter_followers,subreddit_url,reddit_avg_post_48h,reddit_avg_comments_48h,reddit_subscribers,reddit_active_accounts_48h,alexa_rank,bing_matches,coin_sentiment,coin_coingecko_score,coin_developer_score,coin_community_score,coin_liquidity_score,coin_public_interest_score,github_repos,github_forks,github_stars,github_subscribers,github_issues,github_closed_issues,github_pr_merged,github_pr_contributers,github_additions_4w,github_deletions_4w,github_commits_4w
0,bitcoin,SHA-256,[Cryptocurrency],,2009-01-03,,,,,,"[http://www.bitcoin.org, , ]","[https://blockchair.com/bitcoin/, https://btc.com/, https://btc.tokenview.com/, , ]","[https://bitcointalk.org/, , ]",bitcoin,bitcoins,2682026,https://www.reddit.com/r/Bitcoin/,6.583,6.583,3054626,9452,9440.0,None,52.41,81.317,98.883,75.318,100.038,0.381,"[https://github.com/bitcoin/bitcoin, https://github.com/bitcoin/bips]",28996,54527,3816,6274,5692,8678,729,5220.0,-5205.0,381
1,ethereum,Ethash,[Smart Contract Platform],,2015-07-30,2014-07-20T00:00:00.000Z,2014-09-01T00:00:00.000Z,A decentralized platform for applications,1,0.00074794,"[https://www.ethereum.org/, , ]","[https://etherscan.io/, https://ethplorer.io/, https://blockchair.com/ethereum, https://eth.tokenview.com/, https://hecoinfo.com/token/0x64ff637fb478863b7468bc97d30a5bf3a428a1fd]","[https://forum.ethereum.org/, , ]",ethereum,ethereumproject,1346648,https://www.reddit.com/r/ethereum,6.727,6.727,999142,2659,8793.0,None,63.75,78.268,97.236,65.118,100.782,0.473,"[https://github.com/ethereum/go-ethereum, https://github.com/ethereum/py-evm, https://github.com/ethereum/aleth, https://github.com/ethereum/web3.py, https://github.com/ethereum/solidity, https://github.com/ethereum/sharding, https://github.com/ethere...",11276,30935,2108,5618,5423,3766,531,9935.0,-4824.0,58
2,tether,None,"[USD Stablecoin, Stablecoins]",,None,,,,,,"[https://tether.to/, , ]","[https://blockchair.com/bitcoin/omni/property/31, https://www.omniexplorer.info/asset/31, https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7, https://ethplorer.io/address/0xdac17f958d2ee523a2206206994597c13d831ec7, https://tronscan....","[, , ]",Tether_to,tether.to,122013,None,0.000,0.000,0,0,74251.0,None,68.78,42.054,0.000,10.771,107.145,0.066,[],0,0,0,0,0,0,0,NaN,NaN,0
3,binancecoin,None,"[Centralized Exchange Token (CEX), Binance Smart Chain Ecosystem, Exchange-based Tokens]",,2017-07-08,,,,,,"[https://www.binance.com/, , ]","[https://binance.mintscan.io/, https://explorer.binance.org/, https://bscscan.com, https://etherscan.io/token/0xB8c77482e45F1F44dE1745F52C74426C631bDD52, https://ethplorer.io/address/0xB8c77482e45F1F44dE1745F52C74426C631bDD52]","[, , ]",binance,binanceexchange,4185392,https://www.reddit.com/r/binance,9.364,9.364,480096,3305,888.0,None,65.06,68.888,73.255,64.872,86.742,14.394,"[https://github.com/binance-exchange/binance-official-api-docs, https://github.com/binance-exchange/node-binance-api, https://github.com/binance-exchange/php-binance-api]",2020,3405,498,122,122,55,26,0.0,0.0,0
4,cardano,None,[Smart Contract Platform],,None,,,,,,"[https://www.cardano.org/en/home/, , ]","[https://cardanoexplorer.com/, https://cardanoscan.io/, https://blockchair.com/cardano, https://adaex.org/, https://adastat.net/]","[, , ]",CardanoStiftung,,498080,https://www.reddit.com/r/cardano,6.083,6.083,509602,2220,42419.0,None,71.36,65.752,70.463,59.135,84.797,0.302,"[https://github.com/input-output-hk/cardano-sl, https://github.com/input-output-hk/js-cardano-wasm, https://github.com/input-output-hk/plutus, https://github.com/input-output-hk/rust-cardano, https://github.com/input-output-hk/cardano-chain]",608,3620,445,850,665,1706,80,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [ ]:
result = pd.merge(df, df_cd, how="left", left_on='id', right_on='coin_id')
result

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,price_change_percentage_14d_in_currency,price_change_percentage_1h_in_currency,price_change_percentage_1y_in_currency,price_change_percentage_200d_in_currency,price_change_percentage_24h_in_currency,price_change_percentage_30d_in_currency,price_change_percentage_7d_in_currency,created_date,coin_id,coin_algorithm,coin_categories,coin_country_origin,coin_genesis_date,coin_ico_start_date,coin_ico_end_date,coin_short_desc,coin_base_public_sale_amount,coin_quote_public_sale_amount,coin_website,blockchain_website,coin_forum,twitter,facebook,twitter_followers,subreddit_url,reddit_avg_post_48h,reddit_avg_comments_48h,reddit_subscribers,reddit_active_accounts_48h,alexa_rank,bing_matches,coin_sentiment,coin_coingecko_score,coin_developer_score,coin_community_score,coin_liquidity_score,coin_public_interest_score,github_repos,github_forks,github_stars,github_subscribers,github_issues,github_closed_issues,github_pr_merged,github_pr_contributers,github_additions_4w,github_deletions_4w,github_commits_4w
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/large/bitcoin.png?1547033579,33113.00,6.187379e+11,1.0,6.937161e+11,4.878953e+10,35701.00,31109.000000,-2446.866501,-6.88103,-4.610657e+10,-6.93494,1.873028e+07,2.100000e+07,2.100000e+07,64805.00,-49.02496,2021-04-14T11:54:46.763Z,67.810000,48616.58672,2013-07-06T00:00:00.000Z,None,2021-06-08T19:44:04.454Z,-14.310002,1.906379,240.014958,85.820196,-6.881034,-43.658928,-11.322695,2021-06-08 19:46:23,bitcoin,SHA-256,[Cryptocurrency],,2009-01-03,,,,,,"[http://www.bitcoin.org, , ]","[https://blockchair.com/bitcoin/, https://btc.com/, https://btc.tokenview.com/, , ]","[https://bitcointalk.org/, , ]",bitcoin,bitcoins,2682026.0,https://www.reddit.com/r/Bitcoin/,6.583,6.583,3054626.0,9452.0,9440.0,None,52.41,81.317,98.883,75.318,100.038,0.381,"[https://github.com/bitcoin/bitcoin, https://github.com/bitcoin/bips]",28996.0,54527.0,3816.0,6274.0,5692.0,8678.0,729.0,5220.0,-5205.0,381.0
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/large/ethereum.png?1595348880,2510.58,2.907343e+11,2.0,NaN,5.044825e+10,2731.14,2321.900000,-208.419269,-7.66530,-2.446353e+10,-7.76133,1.162108e+08,NaN,NaN,4356.99,-42.57545,2021-05-12T14:41:48.623Z,0.432979,577753.22700,2015-10-20T00:00:00.000Z,"{'times': 101.0027326057399, 'currency': 'btc', 'percentage': 10100.27326057399}",2021-06-08T19:44:47.199Z,-4.908085,1.718776,927.970281,432.671906,-7.665298,-35.835847,-7.305026,2021-06-08 19:46:23,ethereum,Ethash,[Smart Contract Platform],,2015-07-30,2014-07-20T00:00:00.000Z,2014-09-01T00:00:00.000Z,A decentralized platform for applications,1,0.00074794,"[https://www.ethereum.org/, , ]","[https://etherscan.io/, https://ethplorer.io/, https://blockchair.com/ethereum, https://eth.tokenview.com/, https://hecoinfo.com/token/0x64ff637fb478863b7468bc97d30a5bf3a428a1fd]","[https://forum.ethereum.org/, , ]",ethereum,ethereumproject,1346648.0,https://www.reddit.com/r/ethereum,6.727,6.727,999142.0,2659.0,8793.0,None,63.75,78.268,97.236,65.118,100.782,0.473,"[https://github.com/ethereum/go-ethereum, https://github.com/ethereum/py-evm, https://github.com/ethereum/aleth, https://github.com/ethereum/web3.py, https://github.com/ethereum/solidity, https://github.com/ethereum/sharding, https://github.com/ethere...",11276.0,30935.0,2108.0,5618.0,5423.0,3766.0,531.0,9935.0,-4824.0,58.0
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/large/Tether-logo.png?1598003707,1.01,6.319356e+10,3.0,NaN,1.082460e+11,1.01,0.977026,0.013408,1.34307,8.715291e+08,1.39843,6.246236e+10,6.246236e+10,NaN,1.32,-23.53489,2018-07-24T00:00:00.000Z,0.572521,76.71077,2015-03-02T00

# Parking Lot

In [ ]:
# Get current data (name, price, market, ... including exchange tickers) for a coin
coin_details = cg.get_coin_by_id(id='ethereum', vs_currency='usd')
coin_details

{'additional_notices': [],
 'asset_platform_id': None,
 'block_time_in_minutes': 0,
 'categories': ['Smart Contract Platform'],
 'coingecko_rank': 2,
 'coingecko_score': 78.268,
 'community_data': {'facebook_likes': None,
  'reddit_accounts_active_48h': 2731,
  'reddit_average_comments_48h': 313.75,
  'reddit_average_posts_48h': 7.0,
  'reddit_subscribers': 999488,
  'telegram_channel_user_count': None,
  'twitter_followers': 1346648},
 'community_score': 65.118,
 'country_origin': '',
 'description': {'ar': '',
  'de': 'Ethereum is a <a href="https://www.coingecko.com/en?category_id=29&view=market">smart contract platform</a> that enables developers to build tokens and decentralized applications (dapps). ETH is the native currency for the Ethereum platform and also works as the transaction fees to miners on the Ethereum network.\r\n\r\nEthereum is the pioneer for blockchain based smart contracts. Smart contract is essentially a computer code that runs exactly as programmed without any

In [ ]:
type(coin_details)

dict

In [ ]:
coin_details.keys()

dict_keys(['id', 'symbol', 'name', 'asset_platform_id', 'platforms', 'block_time_in_minutes', 'hashing_algorithm', 'categories', 'public_notice', 'additional_notices', 'localization', 'description', 'links', 'image', 'country_origin', 'genesis_date', 'sentiment_votes_up_percentage', 'sentiment_votes_down_percentage', 'ico_data', 'market_cap_rank', 'coingecko_rank', 'coingecko_score', 'developer_score', 'community_score', 'liquidity_score', 'public_interest_score', 'market_data', 'community_data', 'developer_data', 'public_interest_stats', 'status_updates', 'last_updated', 'tickers'])

In [ ]:
coin_details['id']


'ethereum'

In [ ]:
coin_details['platforms']

{'': '',
 'binance-smart-chain': '0x2170ed0880ac9a755fd29b2688956bd959f933f8',
 'huobi-token': '0x64ff637fb478863b7468bc97d30a5bf3a428a1fd'}

In [ ]:
coin_details['block_time_in_minutes']

0

In [ ]:
coin_details['hashing_algorithm']

'Ethash'

In [ ]:
coin_details['categories']

['Smart Contract Platform']

In [ ]:
coin_details['links']

{'announcement_url': ['', ''],
 'bitcointalk_thread_identifier': 428589,
 'blockchain_site': ['https://etherscan.io/',
  'https://ethplorer.io/',
  'https://blockchair.com/ethereum',
  'https://eth.tokenview.com/',
  'https://hecoinfo.com/token/0x64ff637fb478863b7468bc97d30a5bf3a428a1fd'],
 'chat_url': ['', '', ''],
 'facebook_username': 'ethereumproject',
 'homepage': ['https://www.ethereum.org/', '', ''],
 'official_forum_url': ['https://forum.ethereum.org/', '', ''],
 'repos_url': {'bitbucket': [],
  'github': ['https://github.com/ethereum/go-ethereum',
   'https://github.com/ethereum/py-evm',
   'https://github.com/ethereum/aleth',
   'https://github.com/ethereum/web3.py',
   'https://github.com/ethereum/solidity',
   'https://github.com/ethereum/sharding',
   'https://github.com/ethereum/casper',
   'https://github.com/paritytech/parity']},
 'subreddit_url': 'https://www.reddit.com/r/ethereum',
 'telegram_channel_identifier': '',
 'twitter_screen_name': 'ethereum'}

In [ ]:
coin_details['links']['homepage']

['https://www.ethereum.org/', '', '']

In [ ]:
coin_details['links']['blockchain_site']

['https://etherscan.io/',
 'https://ethplorer.io/',
 'https://blockchair.com/ethereum',
 'https://eth.tokenview.com/',
 'https://hecoinfo.com/token/0x64ff637fb478863b7468bc97d30a5bf3a428a1fd']

In [ ]:
coin_details['links']['official_forum_url']

['https://forum.ethereum.org/', '', '']

In [ ]:
coin_details['links']['twitter_screen_name']

'ethereum'

In [ ]:
coin_details['links']['facebook_username']

'ethereumproject'

In [ ]:
coin_details['links']['subreddit_url']

'https://www.reddit.com/r/ethereum'

In [ ]:
coin_details['links']['repos_url']['github']

['https://github.com/ethereum/go-ethereum',
 'https://github.com/ethereum/py-evm',
 'https://github.com/ethereum/aleth',
 'https://github.com/ethereum/web3.py',
 'https://github.com/ethereum/solidity',
 'https://github.com/ethereum/sharding',
 'https://github.com/ethereum/casper',
 'https://github.com/paritytech/parity']

In [ ]:
coin_details['country_origin']

''

In [ ]:
coin_details['genesis_date']

'2015-07-30'

In [ ]:
coin_details['sentiment_votes_up_percentage']

64.2

In [ ]:
coin_details['sentiment_votes_down_percentage']

35.8

In [ ]:
coin_details['ico_data']

{'accepting_currencies': '',
 'amount_for_sale': None,
 'base_pre_sale_amount': None,
 'base_public_sale_amount': 1.0,
 'bounty_detail_url': '',
 'country_origin': '',
 'description': None,
 'hardcap_amount': None,
 'hardcap_currency': '',
 'ico_end_date': '2014-09-01T00:00:00.000Z',
 'ico_start_date': '2014-07-20T00:00:00.000Z',
 'kyc_required': True,
 'links': {},
 'pre_sale_available': None,
 'pre_sale_end_date': None,
 'pre_sale_ended': False,
 'pre_sale_start_date': None,
 'quote_pre_sale_amount': None,
 'quote_pre_sale_currency': '',
 'quote_public_sale_amount': 0.00074794,
 'quote_public_sale_currency': 'BTC',
 'short_desc': 'A decentralized platform for applications',
 'softcap_amount': None,
 'softcap_currency': '',
 'total_raised': None,
 'total_raised_currency': '',
 'whitelist_available': None,
 'whitelist_end_date': None,
 'whitelist_start_date': None,
 'whitelist_url': ''}

In [ ]:
coin_details['ico_data']['ico_start_date']

'2014-07-20T00:00:00.000Z'

In [ ]:
coin_details['ico_data']['ico_end_date']

'2014-09-01T00:00:00.000Z'

In [ ]:
coin_details['ico_data']['short_desc']

'A decentralized platform for applications'

In [ ]:
coin_details['ico_data']['base_public_sale_amount']

1.0

In [ ]:
coin_details['ico_data']['quote_public_sale_amount']

0.00074794

In [ ]:
coin_details['coingecko_score']

78.268

In [ ]:
coin_details['developer_score']

97.236

In [ ]:
coin_details['community_score']

65.118

In [ ]:
coin_details['liquidity_score']

100.782

In [ ]:
coin_details['public_interest_score']

0.473

In [ ]:
coin_details['community_data']

{'facebook_likes': None,
 'reddit_accounts_active_48h': 2731,
 'reddit_average_comments_48h': 313.75,
 'reddit_average_posts_48h': 7.0,
 'reddit_subscribers': 999488,
 'telegram_channel_user_count': None,
 'twitter_followers': 1346648}

In [ ]:
coin_details['community_data']['facebook_likes']

In [ ]:
coin_details['community_data']['twitter_followers']

1346648

In [ ]:
coin_details['community_data']['reddit_average_posts_48h']

7.0

In [ ]:
coin_details['community_data']['reddit_average_comments_48h']

313.75

In [ ]:
coin_details['community_data']['reddit_subscribers']

999488

In [ ]:
coin_details['community_data']['reddit_accounts_active_48h']

2731

In [ ]:
coin_details['community_data']['telegram_channel_user_count']

In [ ]:
coin_details['developer_data']['forks']

11276

In [ ]:
coin_details['developer_data']['stars']

30935

In [ ]:
coin_details['developer_data']['subscribers']

2108

In [ ]:
coin_details['developer_data']['total_issues']

5618

In [ ]:
coin_details['developer_data']['closed_issues']

5423

In [ ]:
coin_details['developer_data']['pull_requests_merged']

3766

In [ ]:
coin_details['developer_data']['pull_request_contributors']

531

In [ ]:
coin_details['developer_data']['code_additions_deletions_4_weeks']['additions']

9935

In [ ]:
coin_details['developer_data']['code_additions_deletions_4_weeks']['deletions']

-4824

In [ ]:
coin_details['developer_data']['commit_count_4_weeks']

58

In [ ]:
coin_details['public_interest_stats']['alexa_rank']

8793

In [ ]:
coin_details['public_interest_stats']['bing_matches']

In [ ]:
coin_details.keys()

dict_keys(['id', 'symbol', 'name', 'asset_platform_id', 'platforms', 'block_time_in_minutes', 'hashing_algorithm', 'categories', 'public_notice', 'additional_notices', 'localization', 'description', 'links', 'image', 'country_origin', 'genesis_date', 'sentiment_votes_up_percentage', 'sentiment_votes_down_percentage', 'ico_data', 'market_cap_rank', 'coingecko_rank', 'coingecko_score', 'developer_score', 'community_score', 'liquidity_score', 'public_interest_score', 'market_data', 'community_data', 'developer_data', 'public_interest_stats', 'status_updates', 'last_updated', 'tickers'])

# Parking Lot

In [ ]:
# Get historical data (name, price, market, stats) at a given date for a coin
cg_coin_history = cg.get_coin_history_by_id(id='ethereum', date='01-01-2021', localization='false', vs_currency='usd')
cg_coin_history

{'id': 'ethereum',
 'symbol': 'eth',
 'name': 'Ethereum',
 'image': {'thumb': 'https://assets.coingecko.com/coins/images/279/thumb/ethereum.png?1595348880',
  'small': 'https://assets.coingecko.com/coins/images/279/small/ethereum.png?1595348880'},
 'market_data': {'current_price': {'aed': 2712.9459227679636,
   'ars': 62098.96720675696,
   'aud': 959.3518715018406,
   'bch': 2.149101840754451,
   'bdt': 62510.965519007295,
   'bhd': 278.4438133445556,
   'bmd': 738.6169381520413,
   'bnb': 19.756413227674095,
   'brl': 3836.0809299864463,
   'btc': 0.02545829679150563,
   'cad': 940.2593622675483,
   'chf': 652.1514849042125,
   'clp': 524787.0103041892,
   'cny': 4825.16287186585,
   'czk': 15856.480704860396,
   'dkk': 4499.580525528412,
   'dot': 78.49453425770511,
   'eos': 284.40221565391164,
   'eth': 1.0,
   'eur': 604.6554613132835,
   'gbp': 540.2406781370432,
   'hkd': 5726.903360808755,
   'huf': 219310.14127610414,
   'idr': 10308349.971911158,
   'ils': 2373.190994621274,


In [ ]:
# Get historical market data including price, market cap, and 24h volume (granularity auto)
cg.get_coin_market_chart_by_id(id='ethereum', vs_currency='usd', days=30)

{'prices': [[1620187277523, 3290.9178848859738],
  [1620191076327, 3277.5732351213687],
  [1620194515152, 3282.7787530547866],
  [1620198322148, 3272.518565112157],
  [1620201849372, 3351.3356383444107],
  [1620205336625, 3363.862815906287],
  [1620209136340, 3399.382281676815],
  [1620212668967, 3385.0240220741334],
  [1620216387726, 3354.414808417009],
  [1620219726623, 3383.4693627050447],
  [1620223574717, 3383.9674967098617],
  [1620226919066, 3392.7226357930413],
  [1620230702841, 3329.05650581122],
  [1620234280611, 3433.2925233012256],
  [1620237901130, 3412.60274688405],
  [1620241475465, 3419.6221240365926],
  [1620245611995, 3473.7409044426577],
  [1620248479493, 3469.620555958864],
  [1620252077828, 3428.2181541141777],
  [1620255830609, 3521.088386686321],
  [1620259470107, 3527.87795953073],
  [1620263129706, 3480.4997668489546],
  [1620266583857, 3484.6467897250045],
  [1620270258886, 3492.264680838447],
  [1620273931503, 3467.676500222258],
  [1620277298202, 3446.282991

In [ ]:
# Get coin's OHLC (beta)
cg.get_coin_ohlc_by_id(id='ethereum', vs_currency='usd', days=30)

[[1620201600000, 3290.92, 3290.92, 3272.52, 3272.52],
 [1620216000000, 3351.34, 3399.38, 3351.34, 3385.02],
 [1620230400000, 3354.41, 3392.72, 3354.41, 3392.72],
 [1620244800000, 3329.06, 3433.29, 3329.06, 3419.62],
 [1620259200000, 3473.74, 3521.09, 3428.22, 3521.09],
 [1620273600000, 3527.88, 3527.88, 3480.5, 3492.26],
 [1620288000000, 3467.68, 3467.68, 3408.76, 3408.76],
 [1620302400000, 3445.75, 3492.6, 3440.87, 3492.6],
 [1620316800000, 3520.97, 3520.97, 3487.57, 3516.64],
 [1620331200000, 3540.31, 3574.82, 3458.6, 3458.6],
 [1620345600000, 3496.98, 3534.86, 3468.69, 3534.86],
 [1620360000000, 3495.08, 3512.88, 3440.33, 3440.33],
 [1620374400000, 3442.69, 3453.17, 3416.36, 3453.17],
 [1620388800000, 3447.46, 3467.29, 3442.26, 3467.29],
 [1620403200000, 3451.75, 3509.61, 3451.75, 3509.61],
 [1620417600000, 3571.37, 3571.37, 3528.25, 3528.25],
 [1620432000000, 3518.2, 3518.2, 3451.98, 3451.98],
 [1620446400000, 3493.53, 3538.86, 3493.53, 3538.54],
 [1620460800000, 3530.55, 3566.67, 

### Get Exchanges

In [ ]:
# List all exchanges
exchanges = cg.get_exchanges_list()

columns = ['id', 'name', 'year_established', 'country', 'description', 'url', 'image', 'has_trading_incentive', 'trust_score', 'trust_score_rank', 'trade_volume_24h_btc', 
           'trade_volume_24h_btc_normalized']
exchanges_df = pd.DataFrame(data=exchanges, columns=columns)
exchanges_df

,id,name,year_established,country,description,url,image,has_trading_incentive,trust_score,trust_score_rank,trade_volume_24h_btc,trade_volume_24h_btc_normalized
0,binance,Binance,2017.0,Cayman Islands,,https://www.binance.com/,https://assets.coingecko.com/markets/images/52/small/binance.jpg?1519353250,False,10,1,840986.471825,840986.471825
1,gdax,Coinbase Exchange,2012.0,United States,,https://www.coinbase.com,https://assets.coingecko.com/markets/images/23/small/Coinbase_Coin_Primary.png?1621471875,False,10,2,100841.142060,100841.142060
2,kraken,Kraken,2011.0,United States,,https://r.kraken.com/Q1m9x,https://assets.coingecko.com/markets/images/29/small/kraken.jpg?1584251255,False,10,3,43106.695089,43106.695089
3,gate,Gate.io,NaN,Hong Kong,,https://gate.io/,https://assets.coingecko.com/markets/images/60/small/gateio.jpg?1519793974,False,10,4,33227.172328,33227.172328
4,binance_us,Binance US,2019.0,United States,,https://www.binance.us/en,https://assets.coingecko.com/markets/images/469/small/Binance.png?1568875842,False,10,5,29336.750938,29336.750938
5,bitfinex,Bitfinex,2014.0,British Virgin Islands,,https://www.bitfinex.com,https://assets.coingecko.com/markets/images/4/small/BItfinex.png?1615895883,False,10,6,22008.576985,22008.576985
6,latoken,LATOKEN,2017.0,Cayman Islands,,https://latoken.com/,https://assets.coingecko.com/markets/images/124/small/LA_token.png?1605773251,False,10,7,8376.230404,8376.230404
7,crypto_com,Crypto.com,2019.0,Cayman Islands,"Crypto.com Exchange is the best place to trade crypto, with deep liquidity, low fees and best execution prices, users can trade major cryptocurrencies like Bitcoin, Ethereum, and many more and receive great CRO-powered rewards",https://crypto.com/exchange,https://assets.coingecko.com/markets/images/589/small/crypto_com.jpg?1602499898,False,10,8,6137.663830,6137.663830
8,gemini,Gemini,2014.0,United States,,https://gemini.sjv.io/bZ49k,https://assets.coingecko.com/markets/images/50/small/gemini.png?1605704107,False,10,9,6023.391121,6023.391121
9,huobi,Huobi Global,2013.0,Seychelles,,https://www.huobi.com,https://assets.coingecko.com/markets/images/25/small/1481589873352_.pic_hd.jpg?1589962155,False,9,10,243819.180980,243819.180980


In [ ]:
# List all supported markets id and name (no pagination required)
exchange_names = cg.get_exchanges_id_name_list()

exchange_names_df = pd.DataFrame(exchange_names, columns=['id', 'name'])
exchange_names_df

,id,name
0,aave,Aave
1,aax,AAX
2,aax_futures,AAX Futures
3,abcc,ABCC
4,abit,Abit
5,acdx,ACDX
6,acdx_futures,ACDX Futures
7,aex,AEX
8,allbit,Allbit
9,allcoin,Allcoin


### Get Trending

In [ ]:
trending = cg.get_search_trending()

keys = ['id', 'coin_id', 'name', 'symbol', 'market_cap_rank', 'thumb', 'small', 'large', 'slug', 'price_btc', 'score']

trending['coins']

[{'item': {'id': 'automata',
   'coin_id': 15985,
   'name': 'Automata',
   'symbol': 'ATA',
   'market_cap_rank': 184,
   'thumb': 'https://assets.coingecko.com/coins/images/15985/thumb/ATA.jpg?1622535745',
   'small': 'https://assets.coingecko.com/coins/images/15985/small/ATA.jpg?1622535745',
   'large': 'https://assets.coingecko.com/coins/images/15985/large/ATA.jpg?1622535745',
   'slug': 'automata',
   'price_btc': 3.7813734289489976e-05,
   'score': 0}},
 {'item': {'id': 'matic-network',
   'coin_id': 4713,
   'name': 'Polygon',
   'symbol': 'MATIC',
   'market_cap_rank': 17,
   'thumb': 'https://assets.coingecko.com/coins/images/4713/thumb/matic___polygon.jpg?1612939050',
   'small': 'https://assets.coingecko.com/coins/images/4713/small/matic___polygon.jpg?1612939050',
   'large': 'https://assets.coingecko.com/coins/images/4713/large/matic___polygon.jpg?1612939050',
   'slug': 'polygon',
   'price_btc': 4.316423781941907e-05,
   'score': 1}},
 {'item': {'id': 'shiba-inu',
   'coi